Importing libraries

In [38]:
import os
from sqlite3 import Row

import boto3
import gzip
import json
from dotenv import load_dotenv
from io import BytesIO
from pyspark.sql import SparkSession

# Load environment variables
load_dotenv()

True

Connecting to S3 bucket

In [3]:
# Creating Boto3 Session
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')
aws_bucket_name = os.getenv('AWS_BUCKET_NAME')

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)
print(session)

prefix = 'DE/monthly/'

# Create an S3 client
s3 = boto3.client('s3')


Session(region_name='eu-central-1')


Outputting the list of files in the bucket

In [4]:
# Get the list of objects in the S3 bucket
response = s3.list_objects_v2(Bucket=aws_bucket_name, Prefix=prefix, Delimiter='/')
print(response)


{'ResponseMetadata': {'RequestId': 'EJWT9W6VARV7732Q', 'HostId': 'w2qLZwcqv+R3liIf7zYugd84UA/PLAscmiEEvbtwIePO3b007XbiQ7TjQO06oW9epkgfdqUoxNFZlMavXHbw1A==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'w2qLZwcqv+R3liIf7zYugd84UA/PLAscmiEEvbtwIePO3b007XbiQ7TjQO06oW9epkgfdqUoxNFZlMavXHbw1A==', 'x-amz-request-id': 'EJWT9W6VARV7732Q', 'date': 'Wed, 19 Jul 2023 22:18:31 GMT', 'x-amz-bucket-region': 'eu-central-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'IsTruncated': False, 'Name': 'jobfeed-data-feeds', 'Prefix': 'DE/monthly/', 'Delimiter': '/', 'MaxKeys': 1000, 'CommonPrefixes': [{'Prefix': 'DE/monthly/2020-06/'}, {'Prefix': 'DE/monthly/2020-07/'}, {'Prefix': 'DE/monthly/2020-08/'}, {'Prefix': 'DE/monthly/2020-09/'}, {'Prefix': 'DE/monthly/2020-10/'}, {'Prefix': 'DE/monthly/2020-11/'}, {'Prefix': 'DE/monthly/2020-12/'}, {'Prefix': 'DE/monthly/2021-01/'}, {'Prefix': 'DE/monthly/2021-02/'}, {'Prefix': 'DE/monthl

Downloading the files from the bucket

In [5]:
#Number of Months to download
months = 1
# Number of files per month to download
files_per_month = 1
# current project directory parent path
ROOT_DIR = os.path.abspath(os.pardir)


# Get the list of subfolders in the S3 bucket
subfolders = [obj['Prefix'] for obj in response['CommonPrefixes']]
# Get the last N subfolders - N = months of data to download
subfolders = subfolders[-months:]


filesToLoadInDF = []
# Download files from each subfolder
for subfolder in subfolders:
    # Get the list of files in the subfolder
    response = s3.list_objects_v2(Bucket=aws_bucket_name, Prefix=subfolder)
    # Get the file paths
    files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.jsonl.gz')]
    # Only get the first N files
    files = files[:files_per_month]

    # filesToLoadInDF = [filesToLoadInDF.append(f) for f in files]

    # Create the folder in your local machine
    folder = ROOT_DIR + "/data/raw/" + aws_bucket_name + "/" + subfolder
    if not os.path.exists(folder):
        os.makedirs(folder)
    # Download and extract each file
    for file in files:
        filename = file.rsplit("/", 1)[1]
        print('Downloading file {}...'.format(filename))
        print(subfolder + filename)
        print(folder + filename)

        # Download and Save the file
        s3.download_file(Filename=folder + filename, Bucket=aws_bucket_name, Key=subfolder + filename)

        locaFilePath = os.path.join(folder + filename)
        localExtractedFilePath = os.path.join(folder + filename[:-3])
        print(localExtractedFilePath)
        filesToLoadInDF.append(localExtractedFilePath)
        # Extract the data from the gzipped file
        with gzip.open(locaFilePath, 'rb') as gz_file, open(localExtractedFilePath, 'wb') as extract_file:
            extract_file.write(gz_file.read())

        # Delete the gzipped file
        os.remove(locaFilePath)

DE/monthly/2023-06/jobs.0.jsonl.gz
/home/vboxuser/Documents/PycharmProjects/Bigdata-Processing-pipeline/data/raw/jobfeed-data-feeds/DE/monthly/2023-06/jobs.0.jsonl.gz
/home/vboxuser/Documents/PycharmProjects/Bigdata-Processing-pipeline/data/raw/jobfeed-data-feeds/DE/monthly/2023-06/jobs.0.jsonl


Creating the Spark Session

In [6]:
spark = SparkSession.builder \
    .master("local") \
    .appName("DE-Project") \
    .getOrCreate()

print(filesToLoadInDF)
df = spark.read.json(filesToLoadInDF)  # Use the extracted file paths here
df.show()

23/07/20 00:23:14 WARN Utils: Your hostname, Ahmed-Ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/20 00:23:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/20 00:23:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


['/home/vboxuser/Documents/PycharmProjects/Bigdata-Processing-pipeline/data/raw/jobfeed-data-feeds/DE/monthly/2023-06/jobs.0.jsonl']


23/07/20 00:25:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------------+--------------------+-----------------------+--------------------+--------------------+----------------+----------------------+---------------------------+----------------+--------------------+--------------------+--------------------+-----------------------+--------------------+------------------------+---------------------+----------------------+--------------------+----------+--------------------+--------+---------------+--------------------+--------------------+--------------------+---------------------+-------------------+---------------+-------+--------------------+-------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------+--------------------+-------------+--------------------+------------------------+---------------------+--------------------+---------------------+------------------------+---------------------+--------------------------+--------------

In [7]:
df.printSchema()


root
 |-- advertiser_contact_person: string (nullable = true)
 |-- advertiser_email: string (nullable = true)
 |-- advertiser_house_number: string (nullable = true)
 |-- advertiser_location: string (nullable = true)
 |-- advertiser_name: string (nullable = true)
 |-- advertiser_phone: string (nullable = true)
 |-- advertiser_postal_code: string (nullable = true)
 |-- advertiser_reference_number: string (nullable = true)
 |-- advertiser_spend: long (nullable = true)
 |-- advertiser_street: string (nullable = true)
 |-- advertiser_type: struct (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- value: long (nullable = true)
 |-- advertiser_website: string (nullable = true)
 |-- application_description: string (nullable = true)
 |-- apply_url: string (nullable = true)
 |-- available_contact_fields: string (nullable = true)
 |-- candidate_description: string (nullable = true)
 |-- conditions_description: string (nullable = true)
 |-- contract_type: struct (nullable = true

In [9]:
import pyspark.sql.types as T

def change_column_types(df):
  """Changes the column types of the dataframe."""

  # Change the data types of the columns.
  df = df.withColumn("job_id", df["job_id"].cast(T.StringType()))
  df = df.withColumn("posting_count", df["posting_count"].cast(T.LongType()))
  df = df.withColumn("source_website_count", df["source_website_count"].cast(T.LongType()))
  df = df.withColumn("date", df["date"].cast(T.DateType()))
  df = df.withColumn("expiration_date", df["expiration_date"].cast(T.DateType()))
  df = df.withColumn("duration", df["duration"].cast(T.LongType()))
  df = df.withColumn("salary", df["salary"].cast(T.LongType()))
  df = df.withColumn("salary_from", df["salary_from"].cast(T.LongType()))
  df = df.withColumn("salary_to", df["salary_to"].cast(T.LongType()))
  df = df.withColumn("experience_years_from", df["experience_years_from"].cast(T.LongType()))
  df = df.withColumn("experience_years_to", df["experience_years_to"].cast(T.LongType()))
  df = df.withColumn("hours_per_week_from", df["hours_per_week_from"].cast(T.LongType()))
  df = df.withColumn("hours_per_week_to", df["hours_per_week_to"].cast(T.LongType()))

  # Return the dataframe with the changed column types.
  return df

df_copy = df
df_correctedTypes = change_column_types(df_copy)

In [10]:
df_correctedTypes.printSchema()

root
 |-- advertiser_contact_person: string (nullable = true)
 |-- advertiser_email: string (nullable = true)
 |-- advertiser_house_number: string (nullable = true)
 |-- advertiser_location: string (nullable = true)
 |-- advertiser_name: string (nullable = true)
 |-- advertiser_phone: string (nullable = true)
 |-- advertiser_postal_code: string (nullable = true)
 |-- advertiser_reference_number: string (nullable = true)
 |-- advertiser_spend: long (nullable = true)
 |-- advertiser_street: string (nullable = true)
 |-- advertiser_type: struct (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- value: long (nullable = true)
 |-- advertiser_website: string (nullable = true)
 |-- application_description: string (nullable = true)
 |-- apply_url: string (nullable = true)
 |-- available_contact_fields: string (nullable = true)
 |-- candidate_description: string (nullable = true)
 |-- conditions_description: string (nullable = true)
 |-- contract_type: struct (nullable = true

Postgres Connection

In [36]:
# Postgres Connection
import psycopg2
pgconn = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    user=os.getenv('POSTGRES_USER'),
    database=os.getenv('POSTGRES_DB'),
    password=os.getenv('POSTGRES_PASSWORD'),
    port=os.getenv('POSTGRES_PORT')
)
pgconn

<connection object at 0x7ff012b39bc0; dsn: 'user=airflow password=xxx dbname=job_ads_db host=localhost port=5432', closed: 0>

In [42]:
# craete a new database
pgconn.autocommit = True
pgcursor = pgconn.cursor()
# drop_db_query = '''DROP DATABASE IF EXISTS job_ads_db;'''
# pgcursor.execute(drop_db_query)
# create_db_query = '''CREATE DATABASE job_ads_db;'''
# pgcursor.execute(create_db_query)

create_table_query = '''CREATE TABLE  IF NOT EXISTS tk_2022_07 (
  job_id char(32) NULL,
  posting_count int NULL,
  source_website_count int NULL,
  date date NULL,
  sequence_number bigint NULL,
  expiration_date date NULL,
  expired boolean NULL,
  duration int NULL,
  source_url varchar(255) NULL,
  source_website varchar(255) NULL,
  source_type char(2) NULL,
  duplicate boolean NULL,
  first_posting boolean NULL,
  posting_id char(32) NULL,
  duplicate_on_jobsite boolean NULL,
  via_intermediary boolean NULL,
  language char(3) NULL,
  job_title varchar(255) NULL,
  profession varchar(4) NULL,
  profession_group varchar(4) NULL,
  profession_class varchar(4) NULL,
  profession_isco_code varchar(10) NULL,
  location char(5) NULL,
  location_name varchar(255) NULL,
  location_coordinates varchar(25) NULL,
  location_remote_possible boolean NULL,
  region char(2) NULL,
  education_level char(2) NULL,
  advertiser_name varchar(255) NULL,
  advertiser_type char(2) NULL,
  advertiser_street varchar(255) NULL,
  advertiser_postal_code varchar(15) NULL,
  advertiser_location varchar(255) NULL,
  advertiser_phone varchar(255) NULL,
  available_contact_fields varchar(100) NULL,
  organization int NULL,
  organization_name varchar(255) NULL,
  organization_industry char(2) NULL,
  organization_activity varchar(10) NULL,
  organization_size char(2) NULL,
  organization_address varchar(255) NULL,
  organization_street_number varchar(100) NULL,
  organization_postal_code char(5) NULL,
  organization_location char(5) NULL,
  organization_location_name varchar(255) NULL,
  organization_region char(2) NULL,
  contract_type char(2) NULL,
  working_hours_type char(1) NULL,
  hours_per_week_from int NULL,
  hours_per_week_to int NULL,
  employment_type char(1) NULL,
  full_text text NULL,
  job_description text NULL,
  candidate_description text NULL,
  conditions_description text NULL,
  soft_skills text NULL,
  professional_skills text NULL,
  advertiser_house_number varchar(15) NULL,
  advertiser_email varchar(255) NULL,
  advertiser_website varchar(255) NULL,
  advertiser_contact_person varchar(255) NULL,
  advertiser_reference_number varchar(255) NULL,
  application_description text NULL,
  organization_website varchar(100) NULL,
  employer_description text NULL,
  language_skills text NULL,
  it_skills text NULL,
  organization_linkedin_id varchar(255) NULL,
  organization_national_id varchar(25) NULL,
  experience_years_from int NULL,
  salary int NULL,
  salary_from int NULL,
  salary_to int NULL,
  experience_years_to int NULL
); '''

pgcursor.execute(create_table_query)
pgcursor.close()


In [39]:
pycursor = pgconn.cursor()
drop_table_query = '''DROP TABLE IF EXISTS tk_2022_07;'''
pycursor.execute(drop_table_query)